In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import logging
import os
from pathlib import Path

import ase
import numpy as np

from mcmc.system import SurfaceSystem
from mcmc.utils.misc import get_atoms_batch

np.set_printoptions(precision=3, suppress=True)

Initialize test slabs and parameters. We are testing the dominant surfaces at different $\mu_{Sr}$ values. Starting from the DL-TiO2 termination at low $\mu_{Sr}$, we have the SL-TiO2 termination at mid $\mu_{Sr}$, and the SL-SrO at high $\mu_{Sr}$.

In [3]:
# Clear jupyter notebook default handler
default_logger = logging.getLogger()
default_logger.handlers.clear()

# Load prepared slabs
offset_data_path = os.path.join(
    "../tutorials",
    "data/SrTiO3_001/nff",
    "offset_data.json",
)

ref_slab_files = [
    "data/SrTiO3_001/O44Sr12Ti16.cif",
    "data/SrTiO3_001/O36Sr12Ti12.cif",
    "data/SrTiO3_001/O40Sr16Ti12.cif",
]

ref_slabs = [ase.io.read(f) for f in ref_slab_files]

In [4]:
# Initialize paths
surface_name = "SrTiO3_001"
run_folder = Path() / surface_name
run_folder.mkdir(parents=True, exist_ok=True)

try:
    with open(offset_data_path, "r") as f:
        offset_data = json.load(f)
except FileNotFoundError as e:
    print("Offset data file not found. Please check you have downloaded the data.")
    raise e

calc_settings = {
    "calc_name": "NFF",
    "optimizer": "BFGS",
    "chem_pots": {"Sr": -2, "Ti": 0, "O": 0},
    "relax_atoms": True,
    "relax_steps": 20,
    "offset": True,
    "offset_data": offset_data,
}

system_settings = {
    "surface_name": surface_name,
    "surface_depth": 1,
    "cutoff": 5.0,
    "near_reduce": 0.01,
    "planar_distance": 1.5,
    "no_obtuse_hollow": True,
    "ads_site_type": "all",
}

Set up NFF Calculator. Here, we are using the same neural network weights from our Zenodo dataset (https://zenodo.org/record/7927039). The ensemble requires an `offset_data.json` file.

In [5]:
import torch
from nff.io.ase_calcs import NeuralFF
from nff.utils.cuda import cuda_devices_sorted_by_free_mem

from mcmc.calculators import EnsembleNFFSurface

DEVICE = f"cuda:{cuda_devices_sorted_by_free_mem()[-1]}" if torch.cuda.is_available() else "cpu"

# requires an ensemble of models in this path and an `offset_data.json` file
nnids = ["model01", "model02", "model03"]
model_dirs = [
    os.path.join(
        "../tutorials",
        "data/SrTiO3_001/nff",
        str(x),
        "best_model",
    )
    for x in nnids
]

models = []
for modeldir in model_dirs:
    m = NeuralFF.from_file(modeldir, device=DEVICE).model
    models.append(m)

nff_surf_calc = EnsembleNFFSurface(models, device=DEVICE)
nff_surf_calc.set(**calc_settings)

cuequivariance or cuequivariance_torch is not available. Cuequivariance acceleration will be disabled.
/home/dux/NeuralForceField/models
offset data: {'bulk_energies': {'O': -0.17747231201, 'Sr': -0.06043637668, 'SrTiO3': -1.470008697358702}, 'stoidict': {'Sr': 0.49995161381315867, 'Ti': -0.0637500349111578, 'O': -0.31241304903276834, 'offset': -11.324476454433157}, 'stoics': {'Sr': 1, 'Ti': 1, 'O': 3}, 'ref_formula': 'SrTiO3', 'ref_element': 'Ti'} is set from parameters


{'calc_name': 'NFF',
 'optimizer': 'BFGS',
 'chem_pots': {'Sr': -2, 'Ti': 0, 'O': 0},
 'relax_atoms': True,
 'relax_steps': 20,
 'offset': True,
 'offset_data': {'bulk_energies': {'O': -0.17747231201,
   'Sr': -0.06043637668,
   'SrTiO3': -1.470008697358702},
  'stoidict': {'Sr': 0.49995161381315867,
   'Ti': -0.0637500349111578,
   'O': -0.31241304903276834,
   'offset': -11.324476454433157},
  'stoics': {'Sr': 1, 'Ti': 1, 'O': 3},
  'ref_formula': 'SrTiO3',
  'ref_element': 'Ti'}}

Test the reference slabs

In [6]:
ref_slab_batches = [
    get_atoms_batch(
        slab,
        system_settings["cutoff"],
        DEVICE,
        props={"energy": 0, "energy_grad": []},
    )
    for slab in ref_slabs
]

ref_surfs = []
for ref_slab_batch in ref_slab_batches:
    ref_surf = SurfaceSystem(
        ref_slab_batch,
        calc=nff_surf_calc,
        system_settings=system_settings,
        save_folder=run_folder,
    )
    ref_surfs.append(ref_surf)

      Step     Time          Energy          fmax
BFGS:    0 20:33:43     -570.127991        0.737414
BFGS:    1 20:33:44     -570.142517        0.640850
BFGS:    2 20:33:45     -570.188354        0.114388
BFGS:    3 20:33:45     -570.188721        0.100358
BFGS:    4 20:33:47     -570.189758        0.011243
BFGS:    5 20:33:47     -570.189758        0.009677
      Step     Time          Energy          fmax
BFGS:    0 20:33:48     -467.525604        0.141613
BFGS:    1 20:33:48     -467.526703        0.132013
BFGS:    2 20:33:49     -467.534088        0.003509
      Step     Time          Energy          fmax
BFGS:    0 20:33:49     -518.694092        0.779158
BFGS:    1 20:33:50     -518.722046        0.667621
BFGS:    2 20:33:50     -518.753479        1.136067
BFGS:    3 20:33:51     -518.766785        0.397730
BFGS:    4 20:33:51     -518.778076        0.257114
BFGS:    5 20:33:52     -518.780823        0.264034
BFGS:    6 20:33:53     -518.782776        0.093498
BFGS:    7 20:33:5

The below should output:
```
energy of reference slab is [35.931]
energy of reference slab is [12.478]
energy of reference slab is [-4.876]
```

In [7]:
for surf in ref_surfs:
    surf_energy = surf.get_surface_energy()
    print(f"energy of reference slab is {surf_energy}")

energy of reference slab is [35.931]
energy of reference slab is [12.478]
energy of reference slab is [-4.876]
